In [ ]:
import pandas as pd
import dateutil.parser as dp
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
import os
import glob
from collections import OrderedDict
import numpy as np
import re

In [ ]:
DATA_PATH = './firmafl-outputs/'
N_RUNS = 10
FULL_DELAY_SECONDS = 159


setups = ['firmafl', 'firmafl-full', 'firmafl-full-afl2.52b']
setups = ['firmafl', 'firmafl-full'] 


palette = {"firmafl": "C0", 'firmafl-full': 'C3', "firmafl-full-afl2.52b": "C1"}

In [ ]:
# Order as in FirmAFL visualization
targets = OrderedDict({
        "9050":   "DIR-815 hwedwig.cgi",
        "9054":   "DIR-817LW hnap",
        "10566":  "DIR-850L hnap",
        "129780": "TV-IP110WN video.cgi",
        "129781": "TV-IP110WN network.cgi",
        "161161": "TEW-632BRP httpd",
        "161160": "TEW-632BRP miniupdnpd",
        "9925":   "DAP-2695 httpd",
        "10853":  "DIR-825 httpd",
        "N/A": "tcapi DSL-3782",
})

In [ ]:
def read_fuzzer_stats(target, setup, run):
    output_dir_name = 'outputs' if setup == 'firmafl' else 'outputs_full'
    with open(f'{DATA_PATH}/{setup}-config-{target}-run-{run}-duration-86400/{output_dir_name}/fuzzer_stats', 'r') as f:
        data = f.read()
    start_time = int(re.findall('start_time\s*: (\d*)', data, re.M)[0])
    crashes = int(re.findall('unique_crashes\s*: (\d*)', data, re.M)[0])

    # we need to adjust the time budget for full runs, as AFL spins for 180 seconds without doing anythin
    if setup != 'firmafl':
        start_time += FULL_DELAY_SECONDS
    return start_time, crashes

def read_crashs_over_time(target, setup, run, start_time):
    output_dir_name = 'outputs' if setup == 'firmafl' else 'outputs_full'
    with open(f'{DATA_PATH}/{setup}-config-{target}-run-{run}-duration-86400/{output_dir_name}/plot_data', 'r') as f:
        data = f.readlines()[1:] # skip description lines
    ret = {0:0} # Zero initialize - there are no crashes at the beginning o ftime
    last_crashes = None
    for l in data:
        fields = l.split(',')
        time = int(fields[0]) - start_time
        crashes = int(fields[-4])
        
        if crashes == last_crashes:
            continue
        ret[time] = crashes
        last_crashes = crashes
    return ret
    
        
# readall data
all_data = {}
for t in targets:
    all_data[t] = {}
    for s in setups:
        all_data[t][s] = {}
        for r in range(1,N_RUNS+1):
            print(f"Processing {t} for {s} (run {r})")
            try:
                start_time, max_crashes = read_fuzzer_stats(t, s, r)
                
                #print(start_time)
           
                data = read_crashs_over_time(t, s, r, start_time)
                data[86399] = max_crashes # last data point
                #print(data)
                all_data[t][s][f'run-{r-1}'] = data
            
            except FileNotFoundError as e:
                print("Fuzzer didn't run!")
                all_data[t][s][f"run-{r-1}"] = {0:0}

In [ ]:
# fill the missing idxs
plot_dfs = {}

for t in targets:
    plot_dfs[t] = {}
    for s in setups:
        plot_dfs[t][s] = {}
        keys = set()
        for _, run in all_data[t][s].items():
            keys |= run.keys()
        
        
        for i in range(N_RUNS):
            new_dict = {k: np.nan for k in keys}
            print(t, s)
            new_dict.update(all_data[t][s][f'run-{i}'])
            #new_dict[86399] = np.nan # add end data point
            new_dict = OrderedDict(sorted(new_dict.items()))
            df = pd.DataFrame(new_dict.items(), columns=['time','crashes'])
            df['run'] = [f'run-{i}'] * len(df)
            df['setup'] = [f'{s}'] * len(df)

            df = df.ffill()
            df = df.astype({"crashes":"int"})
  

            plot_dfs[t][s][f'run-{i}'] = df
    
        

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(16, 12))

i = 0
j = 0

#sns.set(font_scale=1.0, rc={'text.usetex' : True})
sns.set_style("whitegrid")
#sns.set_style("darkgrid")
#print(sns.color_palette("tab10").as_cmap())

for t, tname in targets.items():
    print(f'Plotting {t}, {tname}')
    
    for s in setups:

     
        print(t,s)
        df = pd.concat( [df for df in plot_dfs[t][s].values()])
        df.to_csv('/tmp/a.csv')

        #sns.set_style("darkgrid")
        g = sns.lineplot(ax=axes[j,i],
            x='time',
            y="crashes",
            #style='fuzzer',
            hue='setup',
            palette=palette,
            data=df,
            errorbar=('ci', 95),
            estimator=np.median,
            dashes=True,
            )
                  
        g.set_title(tname)
        g.set(xlabel=None)
        g.set(ylabel=None)
        g.legend_.remove()
        g.set_xlim([1, 24*60*60+1])
        #g.set_ylim(bottom=0)


        
    i = (i+1) % 5
    if i == 0:
        j += 1
    


fig.text(0.54, 0.025, 'Time in Seconds', ha='center', fontsize=18)
fig.text(0.025, 0.55, '# Crashes', va='center', rotation='vertical', fontsize=18)
handles, labels = g.get_legend_handles_labels()
print(labels)


plt.figlegend( handles, labels, ncol=6, labelspacing=0., bbox_to_anchor=(1, 1, -.22, .025) )
plt.tight_layout()
plt.subplots_adjust(bottom=0.09, left=0.09)
plt.savefig('coverage.png')